# Trains and Tests PLDA

In [155]:
#!/usr/bin/env python3.7
import os
import sys
sys.path.insert(0, "/export/c10/lmorove1/PythonLibs/plda")
import plda
import numpy as np
import matplotlib.pyplot as plt
import kaldi_io
import argparse
import pickle
from sklearn.decomposition import PCA

import pandas as pd 

from sklearn.neighbors import NearestNeighbors


#PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import decomposition

In [118]:
DATA_PATH = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/'

!pwd && ls
os.chdir(DATA_PATH)
!pwd && ls

/export/c08/lmorove1/kaldi/egs/beatPDivec/v1
cmd.sh	 conf  exp    mfcc     README.txt  run.sh   sid    utils
cmd.sh~  data  local  path.sh  runFor.sh   run.sh~  steps
/export/c08/lmorove1/kaldi/egs/beatPDivec/v1
cmd.sh	 conf  exp    mfcc     README.txt  run.sh   sid    utils
cmd.sh~  data  local  path.sh  runFor.sh   run.sh~  steps


In [404]:
def pca(sFileTrai, sFileTest, iComponents):
# os.chdir("/export/c10/lmorove1/kaldi/egs/xVecPD/v2_NeuroConc16k")
# sFileTrai="/export/c10/lmorove1/kaldi/egs/xVecPD/v2_NeuroConc16k/exp/sre18nn/xvectors_Training_Fold1/xvector.scp"
# sOut = Folder where to store results 
# iComponents = nb of classes (nb of measurements_id? or nb of subject_ids?)
    dIvecTrai = { key:mat for key,mat in kaldi_io.read_vec_flt_scp(sFileTrai) }
    vTrai= pd.DataFrame((list(dIvecTrai.values())))
    vLTrai = np.array([x[-1] for x in np.array(list(dIvecTrai.keys()))])
    
    # FIXME : For realPD, we need more than -5 (CIS-PD subject_id is 4 characters long)
    # FIXME REAL-PD it's not only int 
    vTraiSubjectId = np.array(([int(x[-5:-1]) for x in np.array(list(dIvecTrai.keys()))]))

    dIvecTest = { key:mat for key,mat in kaldi_io.read_vec_flt_scp(sFileTest) }
    vTest=np.array(list(dIvecTest.values()), dtype=float)
    vLTest=np.array([x[-1] for x in np.array(list(dIvecTest.keys()))])
    vTestSubjectId = np.array([int(x[-5:-1]) for x in np.array(list(dIvecTest.keys()))])
    
    
    # I don't know why i need to reshape. is it normal that I have a 1D array?
    print('before transform : ', vTrai.shape)
    print('before transform : ', vTest.shape)
    #vTrai = vTrai.reshape(-1, 1)
    #vTest = vTrai.reshape(-1, 1)
    #iComponents=60;

    if isinstance(iComponents, str):
            iComponents=int(iComponents)


    pca = PCA(n_components=iComponents, svd_solver='randomized', whiten=True)
    pca.fit(vTrai)


    vTraiPCA=pca.transform(vTrai)
    vTestPCA=pca.transform(vTest)

    print('after transform : ', vTraiPCA.shape)
    print('after transform : ', vTestPCA.shape)
    return vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId

In [405]:
sFileTrai = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/ivectors_Training_Fold0/ivector.scp'
sFileTest = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/ivectors_Testing_Fold0/ivector.scp'
iComponents = 50 


In [408]:
vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId = pca(sFileTrai, sFileTest, iComponents)


before transform :  (1403, 300)
before transform :  (364, 300)
after transform :  (1403, 50)
after transform :  (364, 50)


In [409]:
# Divide vTrainPCA by patient 

#from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# KNN 
# AND MEANs cosine distance 


global_training_accuracy=[]
global_testing_accuracy=[]

for subject_id in np.unique(vTraiSubjectId):
    print('----- ' + str(subject_id) + '----- ')
    knn = KNeighborsClassifier(n_neighbors=6)
    
    # Filter vTraiPCA and vLTraiPCA for one subject_id
    indices_subject_id = np.where(vTraiSubjectId == subject_id) # HAPPY
    vTraiPCA_subjectid = vTraiPCA[indices_subject_id]
    vLTrai_subjectid = vLTrai[indices_subject_id]
    
    # Filter vTestPCA and vLTestPCA for one subject_id
    indices_subject_id = np.where(vTestSubjectId == subject_id)
    vTestPCA_subjectid = vTestPCA[indices_subject_id]
    vLTest_subjectid = vLTest[indices_subject_id]

    knn.fit(vTraiPCA_subjectid, vLTrai_subjectid)
    
#     y_labels = knn.predict(vTraiPCA_subjectid)
#     display(y_labels)
    print('Training accuracy: ', knn.score(vTraiPCA_subjectid, vLTrai_subjectid))
    global_training_accuracy.append(knn.score(vTraiPCA_subjectid, vLTrai_subjectid))
    print('Testing accuracy: ', knn.score(vTestPCA_subjectid, vLTest_subjectid))
    global_testing_accuracy.append(knn.score(vTestPCA_subjectid, vLTest_subjectid))

print('global_training_accuracy : ', pd.DataFrame(global_training_accuracy).mean()[0])
print('global_testing_accuracy : ', pd.DataFrame(global_testing_accuracy).mean()[0])

# find mean accuracy
# use their scoring function 

----- 1004----- 
Training accuracy:  0.5384615384615384
Testing accuracy:  0.17647058823529413
----- 1006----- 
Training accuracy:  0.7586206896551724
Testing accuracy:  0.625
----- 1007----- 
Training accuracy:  0.5972222222222222
Testing accuracy:  0.5166666666666667
----- 1019----- 
Training accuracy:  0.4166666666666667
Testing accuracy:  0.8888888888888888
----- 1020----- 
Training accuracy:  0.782051282051282
Testing accuracy:  0.9743589743589743
----- 1023----- 
Training accuracy:  0.36904761904761907
Testing accuracy:  0.5
----- 1032----- 
Training accuracy:  0.524822695035461
Testing accuracy:  0.2777777777777778
----- 1034----- 
Training accuracy:  0.53125
Testing accuracy:  0.0
----- 1038----- 
Training accuracy:  0.7151515151515152
Testing accuracy:  0.6904761904761905
----- 1039----- 
Training accuracy:  0.5769230769230769
Testing accuracy:  0.6153846153846154
----- 1043----- 
Training accuracy:  0.5185185185185185
Testing accuracy:  0.14285714285714285
----- 1044----- 
Tr

In [373]:
parameters = {'n_neighbors':[1,3,10,15,25,35,50]}